<a href="https://colab.research.google.com/github/harshitadd/Celestini/blob/master/AirQuality_and_Delhi_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [0]:
!pip install firebase-admin

In [0]:
#Get Images from Firebase Storage
import datetime
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage

# Fetch the service account key JSON file contents
cred = credentials.Certificate("credentials.json")

# Initialize the app with a service account, granting admin privileges
app = firebase_admin.initialize_app(cred, {
    'storageBucket': 'fir-4ca2c.appspot.com',
}, name='storage')

In [0]:
def getImgName(name):
  try:
    return (name.split('/')[2].split(':')[0]  + ":" + name.split('/')[2].split(':')[1])
  except:
    print("Not a valid image name")

def getReading(string):
  try:
    return(string[0].split(':')[0] + ":" + string[0].split(':')[1])
  except:
    pass

In [0]:
import csv
data = []
reading = []
# t = 0
bucket = storage.bucket(app=app)
blobs = bucket.list_blobs()
# for blob in blobs:
#   t +=1
# print(t)


mits = set()
for blob in blobs:
    with open('airveda1.csv') as csv_file:
      csv_reader = csv.reader(csv_file, delimiter=',')
      for row in csv_reader:
        reading = []
        if(getReading(row) == getImgName(blob.name) and getReading(row) not in mits):
          mits.add(getImgName(blob.name))
          blob.download_to_filename('/content/' + str(blob.name.split('/')[-1])+ '.jpeg')
          print(getReading(row))
          img = cv2.imread(str(blob.name.split('/')[-1])+ '.jpeg')
          img = cv2.resize(img,(16, 16))
          x = []
          x.append(np.array(img))
          x.append(row[1])
          x.append(row[2])
          x.append(row[3])
          reading.append(np.array(x))
          data.append(np.array(reading)) 

In [0]:
print(np.shape(data))

(255, 1, 4)


In [0]:
data = np.array(data)
data = np.squeeze(data, axis = 1)
print(data.shape)

(255, 4)


In [0]:
def transmission(img, lt = 230, retMean = False):
  ker = np.ones((3, 3))/9.0
  imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  b, g, r = cv2.split(img)
  hsv = cv2.cvtColor(imgrgb, cv2.COLOR_RGB2HSV)
  _, _, v = cv2.split(hsv)
  v_blur = cv2.filter2D(v, -1, ker)
  _,building_mask = cv2.threshold(v_blur, lt, 255, cv2.THRESH_BINARY)
  sky = cv2.bitwise_and(imgrgb, imgrgb, mask = building_mask)
  sky_mask = cv2.bitwise_not(building_mask, building_mask.copy())
  building = cv2.bitwise_and(imgrgb, imgrgb, mask = sky_mask)
  #f, ax = plt.subplots(2, 1, figsize = (15, 15))
  #ax[0].imshow(imgrgb)
  #ax[1].imshow(sky)
  #ax[2].imshow(building)
  al = max(v_blur.flatten())
  #Airlight Found. Now calculate Transmission Map"
  img_norm = img/float(al)
  b, g, r = cv2.split(img_norm)
  kernel = np.ones((3,3),np.uint8)
  im_new = np.zeros((img.shape[0], img.shape[1]))
  for i in range(img.shape[0]):
    for j in range(img.shape[1]):
      im_new[i][j] = min(b[i][j], g[i][j], r[i][j])
  dcp = cv2.erode(im_new, kernel, iterations = 1)
  #ax[3].imshow(im_new, cmap = 'gray')
  #ax[4].imshow(dcp, cmap = 'gray')
  tr = np.ones((dcp.shape[0], dcp.shape[1]))
  for i in range(dcp.shape[0]):
    for j in range(dcp.shape[1]):
      tr[i][j] = 1 - dcp[i][j]
  #ax[1].imshow(tr, cmap = 'gray')
  if retMean == False:
    return tr
  elif retMean == True:
    return np.mean(tr)

In [0]:
def contrast(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  s = 0
  ss = 0
  mean = np.mean(image)
  x = (-1 * mean * (np.ones(image.shape)))
  ss = np.square(np.add(image, x))
  cont = np.sqrt(np.sum(ss)/float(image.size))
  return cont

In [0]:
import math
def entropy(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  hist = cv2.calcHist( [image.astype('float32')],
              [0], 
              None,
               [256], 
              [0,256] )
  h_norm = 0
  h_norm = cv2.normalize(hist, h_norm)
  ent = 0
  for p in h_norm:
    try:
      ent += p*math.log(p, 2)
    except:
      pass
  return (-1 * ent[0])

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip drive/My\ Drive/croppedimg.zip

Archive:  drive/My Drive/croppedimg.zip
replace 201405071100.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
from sklearn.preprocessing import MinMaxScaler
def createFeatureVector(data, m = 'nn', retMean = False):
  features = []
  features_img = []
  features_oth = []
  ctr = 0
  for i in range(data.shape[0]):
    img = data[i][0]
    tr = transmission(img, retMean)
    #print(min(tr.flatten()), max(tr.flatten()))
    if m == 'nn':
      tr = np.reshape(tr, (16, 16, 1))
      features_img.append(tr)
      ent = entropy(img)
      cont = contrast(img)
      features_oth.append([ent, cont])
      ctr += 1
    elif m == 'ml':
      features.append(np.hstack((tr.flatten(), entropy(img), contrast(img))))
      ctr += 1
    print(ctr)
  s1 = MinMaxScaler()
  s2 = MinMaxScaler()
  
  features_oth = np.array(features_oth)
  #print(features_oth)
  #print(features_oth[:, 0])
  features_oth[1] = np.array(features_oth[1])
  z1 = s1.fit_transform(features_oth[:, 0].reshape(-1, 1))
  print(z1.shape)
  z2 = s2.fit_transform(features_oth[:, 1].reshape(-1, 1))
  print(z2.shape)
  z = np.hstack((z1, z2))
  #print(z)
  if m == 'nn':
    return features_img, z
  elif m == 'ml':
    return features

In [0]:
def genLabels(data):
  labels = []
  for i in range(data.shape[0]):
    #x = []
    #x.append(data[i][1])
    #x.append(data[i][3])   
    labels.append(float(data[i][2]))
  return labels

In [0]:
features = createFeatureVector(data, 'nn')

In [0]:
labels = genLabels(data)

In [0]:
print(np.shape(features[0]))
img_feat = features[0]
oth_feat = features[1]
print(np.shape(img_feat))
print(oth_feat.shape)

In [0]:
# print(oth_feat)
# print(oth_feat[:, 0])
print('Entropy Mean: ', np.mean(oth_feat[:, 0]))
print('Entropy Variance: ', np.var(oth_feat[:, 0]))
print('Contrast Mean: ', np.mean(oth_feat[:, 1]))
print('Contrast Variance: ', np.var(oth_feat[:, 1]))
print('Transmission Mean: ', np.mean(img_feat))
print('Transmission Variance: ', np.var(img_feat))

In [0]:

####################################################################################################################################################
############################################################# DELHI DATASET NEURAL NETWORK #########################################################
####################################################################################################################################################

from keras.models import Model
from keras.layers import Dense, Flatten, Conv2D, Input, BatchNormalization, Concatenate, Dropout, LeakyReLU
model = 0


#Input Layers
inp_img = Input(shape=(16, 16, 1, ), dtype='float32', name = 'img_inp')
inp_oth = Input(shape = (2, ), dtype = 'float32', name = 'oth_inp')

#1st Conv + Dropout + BatchNorm
conv1 = Conv2D(8, kernel_size = 3, strides = 2, name = 'conv1', padding = 'same')(inp_img)
relu1 = LeakyReLU(alpha = 0.01)(conv1)
# batch_norm1 = BatchNormalization(name = 'batch1', axis = -1, momentum = 0.05)(relu1)
drop1 = Dropout(rate = 0.7, name = 'drop1')(relu1)






#2nd Conv + Dropout + BatchNorm
conv2 = Conv2D(5, kernel_size = 3, strides = 2, name = 'conv2', padding = 'same')(drop1)#batch_norm1
relu2 = LeakyReLU(alpha = 0.01)(conv2)
# batch_norm2 = BatchNormalization(name = 'batch2', axis = -1, momentum = 0.05)(relu2)
drop2 = Dropout(rate = 0.7, name = 'drop2')(relu2)



#Flatten Convolution Maps
flat = Flatten(name = 'flatConv')(drop2)#drop3



#---------------------------------------------------------------------------------------------------------------
#Send in Entropy and Contrast
dense1 = Dense(10, activation = 'relu', name = 'Dense1')(inp_oth)
dropd2 = Dropout(rate = 0.5, name = 'dropd2')(dense1)

#------------------------------------------------------------------------------------------------------------------
#Join CNN and DNN
join = Concatenate()([flat, dropd2])#drop4
dropJoin = Dropout(rate = 0.5, name = 'dropJoin')(join)


#Add Dense Layer on Top
dense3 = Dense(10, activation='relu', name = 'Dense3')(dropJoin)
dense4 = Dense(5, activation='relu', name = 'Dense3')(dense3)

drop7 = Dropout(rate = 0.5, name = 'drop7')(dense4)



#Output Layer
output = Dense(1, name = 'output')(dense3)#batch_norm5

model = Model(inputs=[inp_img, inp_oth], outputs=[output])
model.summary()

In [0]:
from keras.optimizers import Adadelta, Adam, SGD, RMSprop, Nadam
model.compile(optimizer= Nadam(lr = 0.0001), loss= 'mse')

In [0]:
hist = model.fit([img_feat, oth_feat], labels, epochs = 2000, batch_size = 32, validation_split = 0.3, shuffle = True)

In [0]:
f, ax = plt.subplots(1, 1, figsize = (6, 6))
ax.plot(hist.history['loss'])
ax.plot(hist.history['val_loss'])

In [0]:
model.save('newModel182_910_data.hdf5')

In [0]:
layer_outputs = [layer.output for layer in model.layers[1:3]] # Extracts the outputs of the top 12 layers
activation_model = Model(inputs=model.input, outputs=layer_outputs) # Creates a model that will return these outputs, given the model input

In [0]:
#!pip install git+git://github.com/raghakot/keras-vis.git --upgrade --no-deps
activations = activation_model.predict([img_feat, oth_feat])

In [0]:
print(np.shape(activations[1]))

In [0]:
print(img_feat[20].shape)

In [0]:
first_layer_activation = activations[0][17]
plt.imshow(img_feat[17].reshape((32, 32)), cmap = 'gray')
print(first_layer_activation.shape)
f, ax = plt.subplots(4, 4, figsize = (10, 10))
for i in range(16):
  j = int((i)/4)
  x = int(i%4)
  ax[j][x].matshow(first_layer_activation[:, :, i], cmap='viridis')

In [0]:
#model.save("model_231_198.hdf5")

In [0]:
from keras.models import load_model
model_s = load_model('model_341_411.hdf5')
#model_s.layers.remove()
model_s.summary()

In [0]:
from keras.models import load_model



In [0]:
model_s.save_weights('w.hdf5')

In [0]:
hist_s = model_s.fit([img_feat, oth_feat], labels, epochs = 3000, batch_size = 32, validation_split = 0.3, shuffle = True)

In [0]:
model_s.save("model_s_231_200.hdf5")

In [0]:
f, ax = plt.subplots(1, 1, figsize = (6, 6))
ax.plot(hist_s.history['loss'])
ax.plot(hist_s.history['val_loss'])

In [0]:
pred = model.predict([img_test, oth_test])

In [0]:
from sklearn.metrics import r2_score, mean_absolute_error
print(r2_score(pred, labels_test))
print(mean_absolute_error(pred, labels_test))
rmse = (mean_squared_error(pred, labels_test))**0.5
print(rmse)